Packages Used

-scikit allel (conda install -c conda-forge scikit-allel, pip install scikit-allel
              pip install scikit-allel[full] installs the  optional third-party packages
              including scipy, matplotlib, seaborn, pandas, scikit-learn, h5py and zarr)
           
-UMAP (conda install -c conda-forge umap-learn or pip install umap-learn)

-numpy, scikit-sklearn, pandas 

Plotting Libraries

    -Plotly (pip install plotly )

    -cufflinks (pip install cufflinks, integrates plotly and pandas dataframes)


In [1]:
import allel
allel.__version__

'1.2.0'

In [2]:
import os
import numpy as np
import time
os.listdir('.')

['ALL.wgs.nhgri_coriell_affy_6.20140825.genotypes_has_ped.vcf',
 '1_allel_pca.npy',
 'gt_data_for_chrom_Y.npy',
 'gt_data_for_chrom_MT.npy',
 'Y_allel_pca.npy',
 'ALL.wgs.nhgri_coriell_affy_6.20140825.genotypes_has_ped.vcf.gz',
 'affy_samples.20141118.panel',
 'Task.ipynb',
 '1_scikit_pca.npy',
 'Y_scikit_pca.npy',
 '20131219.populations.tsv',
 '.ipynb_checkpoints']

Reading and Parsing the vcf file 

In [ ]:
#NOT USING THIS
from collections import Counter

class snp(object):

    def __init__(self, line, select=False, autosome_only =True):
        """The initialization method takes in a line from the vcf file, as a string, 
        and records the relevant information. 
        line: a string from a vcf file
        select: a list of positions of individuals to be analyzed, where positions run from 0 to 
        nInd-1, the number of individuals
        """ 
        
        split_line = line.split()  #  First break down the line into a list of each field
        
        self.failed = False  # A label that we will set to True if something goes wrong.
        
        if line.startswith('#'):
            self.failed = True
            self.failure_cause = "line was a header line, not a snp"
            return
        
        if len(split_line)<=5:
            self.failed = True
            self.failure_cause = "incorrectly formatted line, should have at least 5 fields " + line
            return
          
        self.chrom = split_line[0]
        if autosome_only:
            if self.chrom not in ["%d" % (i,) for i in range(1,23)]:
                self.failed = True
                self.failure_cause = "not recognized as an autosome while autosome_only set to True"
                return
        
        self.chrom = int(split_line[0]) # Chromosome (numbered)
        self.position = int(split_line[1])  # The coordinates of the snp
        self.rid = split_line[2] # Name/Record ID
        self.ref_allele = split_line[3]
        self.alt_allele = split_line[4] # The alterate allele according to the vcf; also a string 
        # Only accept snps in ACGT. 
        if self.ref_allele not in ["A","C","G","T"] or self.alt_allele not in ["A","C","G","T"]:
            self.failed = True
            self.failure_cause = "ref or alt not in ACGT"
            return
        self.filter = split_line[6]  # See vcf format specifications for the interpretation of 
                                    # the filter field
        if self.filter not in ['PASS', '.'] :  # PASS indicates a SNP that passed all QC filters.
            self.failed = True
            self.failure_cause = self.filter
            return
              
        self.genotype_strings = split_line[9:]

        # Prepare a list that will contain the transformed genotypes. 
        # Since we already know how long the list will be, it makes sense 
        # to create an array of zeros of the same length as self.gtypes, 
        
        self.genotype_array = np.zeros(len(self.genotype_strings), dtype = np.int8)             

        # Count the number of each genotype. 
        # There may be different strings giving the same genotype so we increment the 
        # counts found so far for the genotype by the number of times the  
        # For example, "0/0" and "0\0" give homref, and "0|1" and "1|0" give het
        
        n_missing = 0
        for index,genotype_string in enumerate(self.genotype_strings):
            if genotype_string == './.':
                n_missing +=1 
                self.genotype_array[index]=-1
                continue # missing data will be left as 0
            allele_0 = genotype_string[0] # Get the first allele (as a string)
            allele_1 = genotype_string[2]
            if (allele_0=='1' and allele_1=='1'): # Use rstrip because windows machines will occasionally have extra \n
                self.genotype_array[index]=2
            elif ((allele_0=='0' and allele_1=='1') or (allele_0=='1' and allele_1=='0')):
                self.genotype_array[index]=1   
            elif (allele_0=='0' and allele_1=='0'):
                # The array was initialized to zero, so nothing to do here!
                continue
            else:
                print(("unknown genotype", genotype_string))
                self.failed=True
                self.failedreason="unknown genotype"
                return

# Specify the number of lines to skip to avoid storing every line in memory
number_of_lines_to_skip = 100

genotype_matrix = []  # Will contain our numerical genotype matrix. 
genotype_positions = []
genotype_names = []
x = 0
error_count = 0

with gzip.open(vcf_file,'rt') as f:
    count = 0
    for line in f:
        count+=1
        if count % number_of_lines_to_skip == 0:
            if line.startswith("#") or snp(line).failed:
                if snp(line).failure_cause != "line was a header line, not a snp":
                    error_count += 1
                    if x < 10:
                        print('Failed: ' + snp(line).failure_cause)
                        x+=1
                continue
            
            return_snp = snp(line)
            genotype_matrix.append(return_snp.genotype_array)
            genotype_names.append(return_snp.rid)
            genotype_positions.append([return_snp.chrom, return_snp.position])

In [ ]:
# Could not directly load from a pandas dataframe because it currently does not return calldata/GT field.
#There's a PR though

data_loading_time=time.time()
data=allel.read_vcf('ALL.wgs.nhgri_coriell_affy_6.20140825.genotypes_has_ped.vcf',fields=['CHROM','GT','samples'])
print('TIME TAKEN FOR DATA LOADING: ',time.time()-data_loading_time)

In [ ]:
data # Have loaded only the relevant fields from the file i.e. the sample array, GT and chromosome variants

Data has successfully been loaded.

In [ ]:
# TO get a rough estimate of the run time of the program except the loading part
start_time=time.time()

In [ ]:
chrom_data=data['variants/CHROM']
gt_data=data['calldata/GT']
samples_data=data['samples']

In [ ]:
print(type(chrom_data),type(gt_data),type(samples_data))

In [ ]:
gendata = allel.GenotypeChunkedArray(data['calldata/GT'])

In [ ]:
gendata

In [ ]:
#To get the frequency of each of the chromosome variants and they are presented in a sorted order so makes it easy to access it
import collections
indices=dict(collections.Counter(chrom_data))

In [ ]:
#Function to get the start and end indices of a particular chromosome variant

def get_indices(indices,samp):    
    start_ind=sum(list(indices.values())[:list(indices.keys()).index(samp)])
    end_ind=start_ind+indices[samp]
    return start_ind,end_ind

In [ ]:
indices

In [ ]:
test_chrom='Y' # Specify chromosome which needs to be analysed

start_index,end_index=get_indices(indices,test_chrom) 
print(start_index,end_index)

In [ ]:
gendata_for_chrom=gendata[start_index:end_index]

Loading data for a particular chromosome to analyze 

In [4]:
gendata= allel.GenotypeChunkedArray(np.load('gt_data_for_chrom_Y.npy')) # Loading a saved file of the chromosome MT
# 'gt_data_for_chrom_Y.npy' is uploaded in the repo as well.

In [5]:
gendata_for_chrom=gendata

In [6]:
gendata_for_chrom

<GenotypeChunkedArray shape=(257, 3450, 2) dtype=int8
   nbytes=1.7M
   values=numpy.ndarray>

In [7]:
# Analysing the data to check the alleles distribution in the data
count_all = gendata_for_chrom.count_alleles()[:]

In [8]:
count_all

<AlleleCountsArray shape=(257, 2) dtype=int32>
1124  583
1682   20
1681   29
...
 516 1182
 300 1408
 300 1400

In [9]:
#Removing/ Filtering out all the singletons present in the data i.e. though biallelic singletons dont exist 

filter_out = (count_all.max_allele() == 1) & (count_all[:, :2].min(axis=1) > 1)
gf = gendata_for_chrom.compress(filter_out, axis=0)
gf

<GenotypeChunkedArray shape=(219, 3450, 2) dtype=int8 chunks=(55, 3450, 2)
   nbytes=1.4M cbytes=680.2K cratio=2.2
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1, 'blocksize': 0}
   values=zarr.core.Array>

In [10]:
# Convert data to a 2-D matrix where each cell has the number of non-reference alleles per call.
gn = gf.to_n_alt()
gn.shape

(219, 3450)

In [11]:
gn

<ChunkedArrayWrapper shape=(219, 3450) dtype=int8 chunks=(55, 3450)
   nbytes=737.8K cbytes=180.3K cratio=4.1
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1, 'blocksize': 0}
   values=zarr.core.Array>

One assumption made before doing PCA is that the data features is independent of each other. 
Usually PCA works best when the data is iid but not really sure till what extent is it relevant to 
this data. Something which needs to be looked into

In [12]:
# The allel.pca standardizes the data internally using the patterson algorithm so no need of doing it separately

allel_pca_time=time.time()
coords1, model1 = allel.pca(gn, 100, scaler='patterson')
print('TIME TAKEN FOR ALLEL PCA: ', time.time()-allel_pca_time)

TIME TAKEN FOR ALLEL PCA:  2.2937417030334473


In [13]:
# The number of components can be changed accordingly but i choose to use the scikit PCA as it allows a component 
# for preserving the eigen energy of the matrix.I have tried using both of them to see the results.
# and to check the differences between them

#Standardizing the data and feeding it to the PCA                    
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scikit_pca_time=time.time()
gn=StandardScaler().fit_transform(gn)
pca = PCA(0.99)
gn_pca=pca.fit_transform(gn.T)
print('TIME TAKEN FOR SCIKIT PCA: ',time.time()-scikit_pca_time)

/media/ssd2/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/media/ssd2/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


TIME TAKEN FOR SCIKIT PCA:  2.0965383052825928


In [14]:
#Analysis of scikit PCA ..rough estimate about the distribution
print(pca.explained_variance_ratio_, pca.singular_values_)
print(gn_pca.shape)

[0.5239181  0.14709304 0.12087026 0.07568912 0.0373413  0.02783794
 0.01670433 0.00818776 0.00682061 0.00458631 0.00340257 0.00304268
 0.0026727  0.00226201 0.00212237 0.00178139 0.00130223 0.00106406
 0.00102671 0.00097112 0.00079207 0.00073617] [482.32448061 255.56634603 231.66868698 183.32611152 128.76633449
 111.1798712   86.12360377  60.29624814  55.03251211  45.12728169
  38.8696794   36.75663198  34.4494544   31.69241005  30.69860175
  28.12462402  24.04644991  21.73657761  21.35168181  20.76553052
  18.75383625  18.07990064]
(3450, 22)


In [15]:
coords1.shape

(3450, 100)

In [16]:
import umap

Using the defualt parameters of UMAP as of now. 

In [17]:
#Applying UMAP on both the obtained PCA matrices
#Need to do something like grid search over the parameters of UMAP to find the best ones.

umap_time=time.time()

red_1=umap.UMAP(random_state=42).fit_transform(gn_pca)
red_2=umap.UMAP(random_state=42).fit_transform(coords1)

print('TIME TAKEN FOR UMAP: ',(time.time()-umap_time)/2)

/media/ssd2/anaconda3/envs/fastai/lib/python3.6/site-packages/umap/spectral.py:229: UserWarning: Embedding a total of 43 separate connected components using meta-embedding (experimental)
  n_components
/media/ssd2/anaconda3/envs/fastai/lib/python3.6/site-packages/umap/spectral.py:182: UserWarning: WARNING: spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  "WARNING: spectral initialisation failed! The eigenvector solver\n"
/media/ssd2/anaconda3/envs/fastai/lib/python3.6/site-packages/umap/spectral.py:229: UserWarning: Embedding a total of 24 separate connected components using meta-embedding (experimental)
  n_components
/media/ssd2/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/manifold/spectral_embedding_.py:237: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is 

TIME TAKEN FOR UMAP:  106.97761356830597


In [ ]:
red_1.shape, red_2.shape

In [ ]:
red_1

In [ ]:
red_2

In [19]:
# Saving the reduced data(2 dimensional) in the numpy format
np.save('{}_scikit_pca.npy'.format('Y'),red_1)#should be replaced by test_chrom if using the whole data
np.save('{}_allel_pca.npy'.format('Y'),red_2)

In [20]:
#Reading the data from panel and tsv file to visualize
import pandas as pd

panel_file= pd.read_csv('affy_samples.20141118.panel', sep="\t", header=0)
population_file = pd.read_csv('20131219.populations.tsv', sep="\t", header=0)

In [21]:
population_file

,Population Description,Population Code,Super Population,DNA from Blood,Offspring available from trios,Pilot Samples,Phase1 Samples,Final Phase Samples,Total
0,"Chinese Dai in Xishuangbanna, China",CDX,EAS,no,yes,0.0,0.0,99.0,99.0
1,"Han Chinese in Bejing, China",CHB,EAS,no,no,91.0,97.0,103.0,106.0
2,"Japanese in Tokyo, Japan",JPT,EAS,no,no,94.0,89.0,104.0,105.0
3,"Kinh in Ho Chi Minh City, Vietnam",KHV,EAS,yes,yes,0.0,0.0,101.0,101.0
4,"Southern Han Chinese, China",CHS,EAS,no,yes,0.0,100.0,108.0,112.0
5,Bengali in Bangladesh,BEB,SAS,no,yes,0.0,0.0,86.0,86.0
6,"Gujarati Indian in Houston,TX",GIH,SAS,no,yes,0.0,0.0,106.0,106.0
7,Indian Telugu in the UK,ITU,SAS,yes,yes,0.0,0.0,103.0,103.0
8,"Punjabi in Lahore,Pakistan",PJL,SAS,yes,yes,0.0,0.0,96.0,96.0
9,Sri Lankan Tamil in the UK,STU,SAS,yes,yes,0.0,0.0,103.0,103.0


In [22]:
population_file.drop([26,27,28])

,Population Description,Population Code,Super Population,DNA from Blood,Offspring available from trios,Pilot Samples,Phase1 Samples,Final Phase Samples,Total
0,"Chinese Dai in Xishuangbanna, China",CDX,EAS,no,yes,0.0,0.0,99.0,99.0
1,"Han Chinese in Bejing, China",CHB,EAS,no,no,91.0,97.0,103.0,106.0
2,"Japanese in Tokyo, Japan",JPT,EAS,no,no,94.0,89.0,104.0,105.0
3,"Kinh in Ho Chi Minh City, Vietnam",KHV,EAS,yes,yes,0.0,0.0,101.0,101.0
4,"Southern Han Chinese, China",CHS,EAS,no,yes,0.0,100.0,108.0,112.0
5,Bengali in Bangladesh,BEB,SAS,no,yes,0.0,0.0,86.0,86.0
6,"Gujarati Indian in Houston,TX",GIH,SAS,no,yes,0.0,0.0,106.0,106.0
7,Indian Telugu in the UK,ITU,SAS,yes,yes,0.0,0.0,103.0,103.0
8,"Punjabi in Lahore,Pakistan",PJL,SAS,yes,yes,0.0,0.0,96.0,96.0
9,Sri Lankan Tamil in the UK,STU,SAS,yes,yes,0.0,0.0,103.0,103.0


In [23]:
panel_file

,sample,pop,inphase3
0,HG00096,GBR,1
1,HG00097,GBR,1
2,HG00098,GBR,0
3,HG00099,GBR,1
4,HG00100,GBR,1
5,HG00101,GBR,1
6,HG00102,GBR,1
7,HG00104,GBR,0
8,HG00105,GBR,1
9,HG00106,GBR,1


In [24]:
temp=population_file['Population Code'].tolist()
full_form=population_file['Population Description'].tolist()

final=[]
for i in panel_file['pop']:
    final.append(full_form[temp.index(i)])

In [25]:
final

['British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Scotland',
 'British in England and Sco

In [26]:
#Appending the 2 dimensions and full forms to a single dataframe

#red_1 = scikit pca + umap
#red_2 = allec pca + umap 

panel_file['x1']=red_1[:,0]
panel_file['x2']=red_1[:,1]
panel_file['full_form']=final

In [27]:
panel_file

,sample,pop,inphase3,x1,x2,full_form
0,HG00096,GBR,1,-1.831447,27.933779,British in England and Scotland
1,HG00097,GBR,1,-6.588832,-0.303326,British in England and Scotland
2,HG00098,GBR,0,-1.451613,27.521818,British in England and Scotland
3,HG00099,GBR,1,-7.400087,-0.277066,British in England and Scotland
4,HG00100,GBR,1,-7.845146,-0.163592,British in England and Scotland
5,HG00101,GBR,1,-28.248121,9.109293,British in England and Scotland
6,HG00102,GBR,1,-8.027495,0.553173,British in England and Scotland
7,HG00104,GBR,0,-7.954947,0.116822,British in England and Scotland
8,HG00105,GBR,1,-1.459808,27.669159,British in England and Scotland
9,HG00106,GBR,1,-7.905345,0.453174,British in England and Scotland


Visualizing via Plotly 

In [28]:
import plotly
plotly.__version__

'3.7.0'

In [29]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np
import cufflinks as cf

In [30]:
init_notebook_mode(connected=True)

The plots are interactive. Helps in better analysis

In [31]:
#Unlabelled plot in 2 D
iplot(panel_file.iplot(asFigure=True,
                               kind='scatter',x='x1',y='x2', xTitle='x1',yTitle='x2', text='full_form',title='2-D plot',mode='markers'))

In [32]:
pop_codes=set(panel_file['pop'].tolist())

In [33]:
pop_codes

{'ACB',
 'ASW',
 'BEB',
 'CDX',
 'CEU',
 'CHB',
 'CHS',
 'CLM',
 'ESN',
 'FIN',
 'GBR',
 'GIH',
 'GWD',
 'IBS',
 'ITU',
 'JPT',
 'KHV',
 'LWK',
 'MSL',
 'MXL',
 'PEL',
 'PJL',
 'PUR',
 'STU',
 'TSI',
 'YRI'}

In [34]:
#Labelled plot 2-D

fig = {
    'data': [
        {'x': panel_file.loc[panel_file['pop']==i]['x1'], 'y': panel_file.loc[panel_file['pop']==i]['x2'], 'text': panel_file.loc[panel_file['pop']==i]['full_form'] , 'mode': 'markers', 'name': i}
        for i in pop_codes
    ],
    'layout': {
        'xaxis': {'title': 'x1'},
        'yaxis': {'title': 'x2'}
    }
}
iplot(fig, filename='scatter-plot')

In [ ]:
time.time()-start_time

References:

    
https://umap-learn.readthedocs.io/en/latest/index.html

https://scikit-allel.readthedocs.io/en/stable/

https://scikit-learn.org/stable/

https://plot.ly/


https://www.wikipedia.org/

https://www.youtube.com/channel/UCKbkfKk65PZyRCzUwXOJung

